# Rule Scorer Example

The Rule Scorer is used to generate scores for a set of rules based on a labelled dataset.

## Requirements

To run, you'll need the following:

* A rule set (specifically the binary columns of the rules as applied to a dataset).
* The binary target column associated with the above dataset.

----

## Import packages

In [1]:
from iguanas.rule_scoring import RuleScorer, PerformanceScorer, ConstantScaler
from iguanas.metrics.classification import Precision

import pandas as pd

## Read in data

Let's read in some dummy rules (stored as binary columns) and the target column.

In [2]:
X_rules_train = pd.read_csv(
    'dummy_data/X_rules_train.csv', 
    index_col='eid'
)
y_train = pd.read_csv(
    'dummy_data/y_train.csv', 
    index_col='eid'
).squeeze()
X_rules_test = pd.read_csv(
    'dummy_data/X_rules_test.csv', 
    index_col='eid'
)
y_test = pd.read_csv(
    'dummy_data//y_test.csv', 
    index_col='eid'
).squeeze()

----

## Generate scores

### Set up class parameters

Now we can set our class parameters for the Rule Scorer. Here we pass an instantiated scoring class (which generates the raw scores) and an instantiated scaling class (which scales the scores to be more readable - **this is optional**). The scoring classes are located in the `rule_scoring_methods` module; the scaling classes are located in the `rule_score_scalers` module. **See the class docstrings for more information on each type of scoring/scaling class.**

In this example, we'll use the `PerformanceScorer` class for scoring the rules (based on the precision score) and the `ConstantScaler` class for scaling. **Note that we're using the *Precision* class from the *metrics.classification* module rather than Sklearn's *precision_score* function, as the former is ~100 times faster on larger datasets.**

**Please see the class docstring for more information on each parameter.**

In [3]:
precision_score = Precision()

In [4]:
params = {
    'scoring_class': PerformanceScorer(metric=precision_score.fit),
    'scaling_class': ConstantScaler(limit=-100)
}

### Instantiate class and run fit method

Once the parameters have been set, we can run the `fit` method to generate scores.

In [5]:
rs = RuleScorer(**params)
rs.fit(
    X_rules=X_rules_train, 
    y=y_train
)

### Outputs

The `fit` method does not return anything. See the `Attributes` section in the class docstring for a description of each attribute generated:

In [6]:
rs.rule_scores.head()

fraud-account_number_num_fraud_transactions_per_account_number_1day>=1                                                                      -99
fraud-account_number_num_fraud_transactions_per_account_number_1day>=1&account_number_num_fraud_transactions_per_account_number_30day>=1    -99
fraud-account_number_num_fraud_transactions_per_account_number_1day>=1&order_total>50.87                                                   -100
fraud-account_number_num_fraud_transactions_per_account_number_7day>=1&order_total>50.87                                                   -100
fraud-account_number_num_fraud_transactions_per_account_number_1day>=1&order_total>60.87                                                   -100
dtype: int64

----

## Apply rules to a separate dataset

Use the `transform` method to apply the generated rules to another dataset.

In [7]:
X_scores_test = rs.transform(X_rules=X_rules_test)

### Outputs

The `transform` method returns a dataframe giving the scores of the rules as applied to the dataset.

In [8]:
X_scores_test.head()

fraud-account_number_avg_order_total_per_account_number_1day<=152.2125&account_number_num_fraud_transactions_per_account_number_30day>=1&account_number_num_order_items_per_account_number_30day>=4&is_existing_user_1==True  \
eid                                                                                                                                                                                                                                                 
975-8351797-7122581                                                  0                                                                                                                                                                              
785-6259585-7858053                                                  0                                                                                                                                                                              
057-4039373-1790681                                                  0                                                                                                                                                                              
095-5263240-3834186                                                  0                                                                                                                                                                              
980-3802574-0009480                                                  0                                                                                                                                                                              

                     fraud-account_number_avg_order_total_per_account_number_1day<=22.61&account_number_num_fraud_transactions_per_account_number_30day>=1&account_number_num_fraud_transactions_per_account_number_90day>=1&order_total<=50.87  \
eid                                                                                                                                                                                                                                               
975-8351797-7122581                                                  0                                                                                                                                                                            
785-6259585-7858053                                                  0                                                                                                                                                                            
057-4039373-1790681                                                  0                                                                                                                                                                            
095-5263240-3834186                                                  0                                                                                                                                                                            
980-3802574-0009480                                                  0                                                                                                                                                                            

                     fraud-account_number_avg_order_total_per_account_number_1day<=23.61&account_number_avg_order_total_per_account_number_7day<=52.5425&account_number_num_fraud_transactions_per_account_number_30day>=1&account_number_num_fraud_transactions_per_account_number_lifetime>=1  \
eid                                                                                                                                                                                                                                                                                               
975-8351797-7122581  

----

## Generate rule score and apply them to the training set (in one step)

You can also use the `fit_transform` method to generate scores and apply them to the training set.

In [9]:
X_scores_train = rs.fit_transform(
    X_rules=X_rules_train, 
    y=y_train
)

### Outputs

The `transform` method returns a dataframe giving the scores of the rules as applied to the dataset. See the `Attributes` section in the class docstring for a description of each attribute generated:

In [10]:
rs.rule_scores.head()

fraud-account_number_num_fraud_transactions_per_account_number_1day>=1                                                                      -99
fraud-account_number_num_fraud_transactions_per_account_number_1day>=1&account_number_num_fraud_transactions_per_account_number_30day>=1    -99
fraud-account_number_num_fraud_transactions_per_account_number_1day>=1&order_total>50.87                                                   -100
fraud-account_number_num_fraud_transactions_per_account_number_7day>=1&order_total>50.87                                                   -100
fraud-account_number_num_fraud_transactions_per_account_number_1day>=1&order_total>60.87                                                   -100
dtype: int64

In [11]:
X_scores_train.head()

fraud-account_number_num_fraud_transactions_per_account_number_1day>=1  \
eid                                                                                           
867-8837095-9305559                                                  0                        
974-5306287-3527394                                                  0                        
584-0112844-9158928                                                  0                        
956-4190732-7014837                                                  0                        
349-7005645-8862067                                                  0                        

                     fraud-account_number_num_fraud_transactions_per_account_number_1day>=1&account_number_num_fraud_transactions_per_account_number_30day>=1  \
eid                                                                                                                                                             
867-8837095-9305559                                                  0                                                                                          
974-5306287-3527394                                                  0                                                                                          
584-0112844-9158928                                                  0                                                                                          
956-4190732-7014837                                                  0                                                                                          
349-7005645-8862067                                                  0                                                                                          

                     fraud-account_number_num_fraud_transactions_per_account_number_1day>=1&order_total>50.87  \
eid                                                                                                             
867-8837095-9305559                                                  0                                          
974-5306287-3527394                                                  0                                          
584-0112844-9158928                                                  0                                          
956-4190732-7014837                                                  0                                          
349-7005645-8862067                                                  0                                          

                     fraud-account_number_num_fraud_transactions_per_account_number_7day>=1&order_total>50.87  \
eid                                                                                                             
867-8837095-9305559                                                  0                                          
974-5306287-3527394                                                  0                                          
584-0112844-9158928                                                  0                                          
956-4190732-7014837                                                  0                                          
349-7005645-8862067                                                  0                                          

                     fraud-account_number_num_fraud_transactions_per_account_number_1day>=1&order_total>60.87  \
eid                                                                                                             
867-8837095-9305559                                                  0                                          
974-5306287-3527394                                                  0                                          
584-0112844-9158928                                                  0                                          
956-4190732-7014837                                                  0        

----